#**Maestría en Inteligencia Artificial Aplicada**
##**Curso: Inteligencia Artificial y Aprendizaje Automático**
###Tecnológico de Monterrey
###Prof Luis Eduardo Falcón Morales

## **Adtividad de la Semana 10**
###**Reducción de Dimensionalidad - Descomposición en Valores Singulares (SVD) y Sistemas de Recomendación**


**Nombres y matrículas de los integrantes del equipo:**

*  Rafael J. Mateo C., A01793054 
*  Daniel Chavarria Barrientos A01331204 
*  Enrique Ricardo García Hernández - A01315428
*  Omar Nayib García Sabag - A01793008

En cada sección deberás incluir todas las líneas de código necesarias para responder a cada uno de los ejercicios.

In [415]:
# Incluye aquí todos módulos, librerías y paquetes que requieras.
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

Comencemos definiendo una clase, la cual nos permitirá reutilizar la mayoría del código para este ejercicio.

In [416]:
class Recommendation:
    
    UtMx_ = None
    relational_table_ = None
    model_ = None
    relational_table_ = None
    resultant_matrix_ = None
    
    ''' El constructor recibe la tabla relacional, así como el número de componentes para el modelo.
    El constructor crea un modelo inicial, pero esto puede ser cambiado más adelante si se desea.
    '''
    def __init__ (self, relational_table, num_comps = 10):
        
        self.relational_table_ = relational_table
        self.model_ = TruncatedSVD(n_components=num_comps, random_state=1)    
    
    
    ''' Se establece la matriz de utilidad. Para ello, es necesario pasar la columna con los valores, la columna
        que será el índice y el nombre de la variable que será la columna de la matriz.
    '''
    def set_UtMx (self, values, index, columns, fill_value = 0):
        self.UtMx_ = self.relational_table_.pivot_table(values=values, index=index, columns=columns, fill_value=fill_value)
    
    '''
    Devuelve la matriz de utilidad
    '''
    def get_UtMx(self):
       
        return self.UtMx_
    
    def get_transposed_UtMx(self):
        return self.UtMx_.T

    def get_resultant_matrix(self):
        return  self.model_.fit_transform(self.get_transposed_UtMx())
    
    def get_names_list(self):
        
        return self.get_UtMx().columns
       
    '''
    Imprime la varianza explicada del modelo para el número de componentes especificado.
    De no especificarse un modelo, el método creará uno por default con todos los componentes
    '''
    def print_explained_var(self, num_sv = 10, model = None):
        if (model == None):
            model = TruncatedSVD(n_components=self.get_UtMx().shape[1], random_state=1)
        
        model.fit(self.get_UtMx())
        print('Cantidad de información simplificada con los primeros %d vectores singulares:' % num_sv)
        print('%.1f%%' %  (100 * (1- (model.singular_values_[0:num_sv]).sum() / (model.singular_values_).sum())))
        
    def get_item_index(self,item):
        return list(self.get_names_list()).index(item)

    def get_corr_mx(self):
        return np.corrcoef(self.resultant_matrix_)
    
    def create_model(self, num_sv = 10):
        self.model_ = TruncatedSVD(n_components=num_sv, random_state=1)
        return self.model_
        
    def get_model(self):
        return self.model_ 
        
    def fit_transform(self):
        self.resultant_matrix_= self.model_.fit_transform(self.get_transposed_UtMx())
        return self.resultant_matrix_

    '''
    Obtiene el top K de las recomendaciones.
    '''
    def get_TOP_k(self, item, k = 10):
        
        index = self.get_item_index(item)
        
        corr_dicc = {'Restaurantes':self.get_names_list(), 'Correlación':self.get_corr_mx()[index]}
        corr_df = pd.DataFrame(corr_dicc)
        return corr_df[corr_df['Restaurantes'] != item].sort_values(by='Correlación', ascending = False).head(k)
  
    def get_recommendations (self, item, lower_bound = 0.95, upper_bound = 0.99):

        names = self.get_names_list()
        
        item_id = list(names).index(item)
        
        corr_recomienda = self.get_corr_mx()[item_id]
        return list(names[(corr_recomienda > lower_bound) & (corr_recomienda < upper_bound)])

'''
Compara dos recomendaciones e imprime el % de coincidencia
'''
def compare_recommendations(original, target):
    org = original['Restaurantes'].to_numpy()
    tg = target['Restaurantes'].to_numpy()
    
    intersect = np.intersect1d(org, tg)
    
    print (f'Las recomendaciones coinciden en un {(len(intersect)/len(original))*100}%')

#**Ejercicio-1.** 

Procedemos a importar los datos

In [417]:
df_rating = pd.read_csv('data/rating_final.csv')
df_geoplaces = pd.read_csv('data/geoplaces2.csv', encoding='latin-1')

Revisamos que se hayan importado correctamente

In [418]:
df_rating.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [419]:
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


Se construye la tabla relacional

In [420]:
tabla_relacional = pd.merge(df_rating, df_geoplaces, on='placeID')   # dataFrame

tabla_relacional.head(10)

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
5,U1116,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
6,U1132,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
7,U1120,135085,0,0,0,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
8,U1036,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
9,U1084,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


Revisamos la información de la tabla relacional

In [421]:
tabla_relacional.shape;

In [422]:
tabla_relacional.head()

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


Ahora generamos la tabla de utilidades

In [423]:
recommendation = Recommendation(tabla_relacional)


recommendation.set_UtMx(values='rating', index='userID', columns='name', fill_value=0)
UtMx = recommendation.get_UtMx()


Revisamos los primeros 10 renglones y revisamos su tamaño.

In [424]:
UtMx.head(10)

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1006,0,0,0,1,0,0,0,0,0,0,...,1,0,2,0,0,0,0,0,0,0
U1007,0,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1008,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1009,0,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [425]:
UtMx.shape

(138, 129)

#**Ejercicio-2.**

Ahora, procederemos a generar un modelo con todos sus componentes posibles, los cuales son 129 (el número total de restaurantes)

In [426]:
num_sv = 129

recommendation.create_model(num_sv)

resultant_matrix = recommendation.fit_transform()
resultant_matrix.shape

(129, 129)

Confirmamos las dimensiones de la matriz de correlación

In [427]:
corr = np.corrcoef(resultant_matrix)
corr.shape

(129, 129)

#**Ejercicio-3.**

Ahora procedemos a generar el top 10 de las recomendaciones, con el modelo que tiene todos sus componentes

In [428]:
recommendation.get_names_list()

Index(['Abondance Restaurante Bar', 'Arrachela Grill', 'Cabana Huasteca',
       'Cafe Chaires', 'Cafeteria cenidet',
       'Cafeteria y Restaurant El Pacifico', 'Carls Jr',
       'Carnitas Mata  Calle 16 de Septiembre',
       'Carreton de Flautas y Migadas', 'Cenaduria El RincÃ³n de Tlaquepaque',
       ...
       'puesto de tacos', 'rockabilly ', 'shi ro ie', 'sirloin stockade',
       'tacos abi', 'tacos de barbacoa enfrente del Tec',
       'tacos de la estacion', 'tacos los volcanes', 'tortas hawai', 'vips'],
      dtype='object', name='name', length=129)

In [429]:
recommendations_all_comps = recommendation.get_TOP_k('Subway', 10)
recommendations_all_comps

,Restaurantes,Correlación
58,Restaurant Teely,0.609521
97,Vips,0.506278
85,Rincon del Bife,0.429824
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.425467
86,Sanborns Casa Piedra,0.417219
41,Mariscos Tia Licha,0.388178
11,Chilis Cuernavaca,0.301090
45,Mikasa,0.301002
64,Restaurant los Pinos,0.285149
37,Log Yin,0.234634


#**Ejercicio-4.**

Ahora generamos las recomendaciones, ajustando los componentes a tal punto que coincida el 90% de las recomendaciones o más.

In [430]:
#Indicamos el número de componentes
num_sv = 10

#Creamos el modelo y lo entrenamos
recommendation.create_model(num_sv)
recommendation.fit_transform()

#generamos las recomendaciones
recommendations_n_comps = recommendation.get_TOP_k('Subway')
recommendations_n_comps

,Restaurantes,Correlación
85,Rincon del Bife,0.999991
46,Paniroles,0.999977
58,Restaurant Teely,0.999956
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.999927
41,Mariscos Tia Licha,0.999906
11,Chilis Cuernavaca,0.999904
45,Mikasa,0.999879
97,Vips,0.999704
37,Log Yin,0.999397
86,Sanborns Casa Piedra,0.999228


In [431]:
#Comparamos las recomendaciones. 
compare_recommendations(recommendations_all_comps, recommendations_n_comps)

Las recomendaciones coinciden en un 90.0%


In [432]:
#Obtenemos la información simplificada con los vectores singulares definidos
recommendation.print_explained_var(num_sv)

Cantidad de información simplificada con los primeros 10 vectores singulares:
75.6%


#**Ejercicio-5.**

Se realiza ahora el analisis utilizando Tacos de Barbacoa 


In [433]:
df_rating.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [434]:
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [435]:
tabla_relacional1 = pd.merge(df_rating, df_geoplaces, on='placeID')   # dataFrame

tabla_relacional1.head(10)

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
5,U1116,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
6,U1132,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
7,U1120,135085,0,0,0,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
8,U1036,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
9,U1084,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


In [436]:
recommendation1 = Recommendation(tabla_relacional1)


recommendation1.set_UtMx(values='rating', index='userID', columns='name', fill_value=0)
UtMx = recommendation1.get_UtMx()
UtMx.head()

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [437]:
num_sv1 = 129

recommendation1.create_model(num_sv1)

resultant_matrix1 = recommendation1.fit_transform()
resultant_matrix1.shape

(129, 129)

In [438]:
recommendations_all_comps1 = recommendation1.get_TOP_k('tacos de barbacoa enfrente del Tec', 10)
recommendations_all_comps1

,Restaurantes,Correlación
128,vips,0.817189
123,tacos abi,0.665305
114,little pizza Emilio Portes Gil,0.653849
92,Taqueria EL amigo,0.437184
118,puesto de gorditas,0.412765
8,Carreton de Flautas y Migadas,0.407795
36,Little Cesarz,0.258479
116,palomo tec,0.195174
25,Hamburguesas La perica,0.191120
100,carnitas mata calle Emilio Portes Gil,0.170939


In [439]:
#Indicamos el número de componentes
num_sv1 = 10

#Creamos el modelo y lo entrenamos
recommendation1.create_model(num_sv1)
recommendation1.fit_transform()

#generamos las recomendaciones
recommendations_n_comps1 = recommendation1.get_TOP_k('tacos de barbacoa enfrente del Tec')
recommendations_n_comps1

,Restaurantes,Correlación
123,tacos abi,0.999956
92,Taqueria EL amigo,0.999901
128,vips,0.999781
114,little pizza Emilio Portes Gil,0.999704
48,Pollo_Frito_Buenos_Aires,0.999520
118,puesto de gorditas,0.999434
101,carnitas_mata,0.999413
116,palomo tec,0.999407
25,Hamburguesas La perica,0.998270
36,Little Cesarz,0.998220


In [440]:
#Comparamos las recomendaciones. 
compare_recommendations(recommendations_all_comps1, recommendations_n_comps1)

Las recomendaciones coinciden en un 80.0%


#**Ejercicio-6.**

Se repite el análisis ahora con la matriz de utilidad en la cual los renglones son los usuarios (user_ID), las columnas los nombres de los restaurantes (name) y las entradas la evaluación dada a la comida (food_rating).

In [441]:
df_rating.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [442]:
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [443]:
tabla_relacional2 = pd.merge(df_rating, df_geoplaces, on='placeID')   # dataFrame

tabla_relacional2.head(10)

,userID,placeID,rating,food_rating,service_rating,latitude,longitude,the_geom_meter,name,address,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,U1077,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
1,U1108,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
2,U1081,135085,1,2,1,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
3,U1056,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
4,U1134,135085,2,1,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
5,U1116,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
6,U1132,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
7,U1120,135085,0,0,0,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
8,U1036,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none
9,U1084,135085,2,2,2,22.150802,-100.98268,0101000020957F00009F823DA6094858C18A2D4D37F9A4...,Tortas Locas Hipocampo,Venustiano Carranza 719 Centro,...,No_Alcohol_Served,not permitted,informal,no_accessibility,medium,?,familiar,f,closed,none


In [444]:
recommendation2 = Recommendation(tabla_relacional2)


recommendation2.set_UtMx(values='food_rating', index='userID', columns='name', fill_value=0)
UtMx = recommendation2.get_UtMx()
UtMx.head()

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,2,2,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [445]:
num_sv2 = 129

recommendation2.create_model(num_sv2)

resultant_matrix2 = recommendation2.fit_transform()
resultant_matrix2.shape

(129, 129)

In [446]:
recommendations_all_comps2 = recommendation2.get_TOP_k('Subway', 10)
recommendations_all_comps2

,Restaurantes,Correlación
58,Restaurant Teely,0.816413
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.468950
97,Vips,0.442148
41,Mariscos Tia Licha,0.415222
85,Rincon del Bife,0.381819
86,Sanborns Casa Piedra,0.365657
55,Restaurant Las Mananitas,0.331707
46,Paniroles,0.312082
29,Kiku Cuernavaca,0.250357
54,Restaurant Familiar El Chino,0.232708


In [447]:
#Indicamos el número de componentes
num_sv2 = 10

#Creamos el modelo y lo entrenamos
recommendation2.create_model(num_sv2)
recommendation2.fit_transform()

#generamos las recomendaciones

recommendations_n_comps2 = recommendation2.get_TOP_k('Subway')
recommendations_n_comps2

,Restaurantes,Correlación
58,Restaurant Teely,1.000000
85,Rincon del Bife,0.999997
97,Vips,0.999996
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.999996
37,Log Yin,0.999996
45,Mikasa,0.999992
41,Mariscos Tia Licha,0.999989
86,Sanborns Casa Piedra,0.999987
46,Paniroles,0.999977
51,Restaurant Bar Coty y Pablo,0.999975


In [448]:
#Comparamos las recomendaciones. 
compare_recommendations(recommendations_all_comps2, recommendations_n_comps2)

Las recomendaciones coinciden en un 70.0%


En este caso el sistema de recomendación va a indicar qué ususarios tienen gustos similares y va a ser sugerencias basándose en la actividad de esos usuarios. Observemos que para el caso de los usuarios que califican como bueno al restaurante *Subway*, también tienen gustos similares para: *Teely, Rincon del Bife, Vips, Log Yin* con más del 85% de correlación.

#**Ejercicio-7.**

In [449]:
df_rating.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


In [450]:
df_geoplaces.head()

,placeID,latitude,longitude,the_geom_meter,name,address,city,state,country,fax,...,alcohol,smoking_area,dress_code,accessibility,price,url,Rambience,franchise,area,other_services
0,134999,18.915421,-99.184871,0101000020957F000088568DE356715AC138C0A525FC46...,Kiku Cuernavaca,Revolucion,Cuernavaca,Morelos,Mexico,?,...,No_Alcohol_Served,none,informal,no_accessibility,medium,kikucuernavaca.com.mx,familiar,f,closed,none
1,132825,22.147392,-100.983092,0101000020957F00001AD016568C4858C1243261274BA5...,puesto de tacos,esquina santos degollado y leon guzman,s.l.p.,s.l.p.,mexico,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,f,open,none
2,135106,22.149709,-100.976093,0101000020957F0000649D6F21634858C119AE9BF528A3...,El Rincón de San Francisco,Universidad 169,San Luis Potosi,San Luis Potosi,Mexico,?,...,Wine-Beer,only at bar,informal,partially,medium,?,familiar,f,open,none
3,132667,23.752697,-99.163359,0101000020957F00005D67BCDDED8157C1222A2DC8D84D...,little pizza Emilio Portes Gil,calle emilio portes gil,victoria,tamaulipas,?,?,...,No_Alcohol_Served,none,informal,completely,low,?,familiar,t,closed,none
4,132613,23.752903,-99.165076,0101000020957F00008EBA2D06DC8157C194E03B7B504E...,carnitas_mata,lic. Emilio portes gil,victoria,Tamaulipas,Mexico,?,...,No_Alcohol_Served,permitted,informal,completely,medium,?,familiar,t,closed,none


In [451]:
recommendation3 = Recommendation(tabla_relacional2)


recommendation3.set_UtMx(values='service_rating', index='userID', columns='name', fill_value=0)
UtMx = recommendation3.get_UtMx()
UtMx.head()

name,Abondance Restaurante Bar,Arrachela Grill,Cabana Huasteca,Cafe Chaires,Cafeteria cenidet,Cafeteria y Restaurant El Pacifico,Carls Jr,Carnitas Mata Calle 16 de Septiembre,Carreton de Flautas y Migadas,Cenaduria El RincÃ³n de Tlaquepaque,...,puesto de tacos,rockabilly,shi ro ie,sirloin stockade,tacos abi,tacos de barbacoa enfrente del Tec,tacos de la estacion,tacos los volcanes,tortas hawai,vips
userID,,,,,,,,,,,,,,,,,,,,,
U1001,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
U1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
U1003,0,0,2,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
U1004,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
U1005,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [452]:
num_sv3 = 129

recommendation3.create_model(num_sv3)

resultant_matrix3 = recommendation3.fit_transform()
resultant_matrix3.shape

(129, 129)

In [453]:
recommendations_all_comps3 = recommendation3.get_TOP_k('Subway', 10)
recommendations_all_comps3

,Restaurantes,Correlación
58,Restaurant Teely,0.772973
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.600161
85,Rincon del Bife,0.518731
97,Vips,0.421200
46,Paniroles,0.365934
11,Chilis Cuernavaca,0.230698
55,Restaurant Las Mananitas,0.226266
21,Giovannis,0.192760
54,Restaurant Familiar El Chino,0.182869
29,Kiku Cuernavaca,0.178740


In [454]:
#Indicamos el número de componentes
num_sv3 = 10

#Creamos el modelo y lo entrenamos
recommendation3.create_model(num_sv3)
recommendation3.fit_transform()

#generamos las recomendaciones

recommendations_n_comps3 = recommendation3.get_TOP_k('Subway')
recommendations_n_comps3

,Restaurantes,Correlación
58,Restaurant Teely,0.999999
60,Restaurant and Bar and Clothesline Carlos N Ch...,0.999998
46,Paniroles,0.999997
97,Vips,0.999980
85,Rincon del Bife,0.999972
41,Mariscos Tia Licha,0.999969
55,Restaurant Las Mananitas,0.999947
37,Log Yin,0.999926
86,Sanborns Casa Piedra,0.999925
42,McDonalds Centro,0.999901


In [455]:
#Comparamos las recomendaciones. 
compare_recommendations(recommendations_all_comps3, recommendations_n_comps3)

Las recomendaciones coinciden en un 60.0%


#**Ejercicio-8.**

* El modelo con la variable 'ratings' fue el que tuvo mejor desempeño, donde el 90% de las recomendaciones coincidieron con apenas 10 componentes. En el caso de los otros modelos, solo coincidieron entre un 60 y 70%, para la misma cantidad de componentes.
* De lo anterior se puede concluir que la variable 'ratings' es un mejor predictor para fines de recomendar restaurantes a los usuarios.



**Bibliografía**

* Deisenroth, M. P., Faisal, A. A., & Ong, C. S. (2020). Mathematics for machine learning. Cambridge University Press
* Géron, A. (2022). Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow. Sebastopol: O’Reilly Media

###**Fin de la Actividad de la semana 10 - Sistemas de Recomendación y Reducción de Dimensionalidad con SVD.**